In [10]:
# ! pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time


In [58]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "./chrome-win64/chrome.exe"  # chrome 경로 설정
options.add_argument("--no--sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage")  # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # (화면크기)해상도 설정, 반응형 웹 대비
# options.add_argument("--disable-gpu") # GPU 비활성화 

# ChromeDriver 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/index.do")
print(driver.title)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 액션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform() # 마우스 올린거 처럼

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(5)

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click() # '지역검색' 을 선택
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a").click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#mCSB_2_container > ul > li:nth-child(1) > a").click()
time.sleep(1)

html = driver.page_source
# print(html)
driver.quit() # 브라우저 종료

Starbucks Korea


In [90]:
# 현재 페이지 정보 가져오기
star_list=[]
soup = bs(html, 'html.parser')
# 본문내용 가져오기
contents = soup.select("#mCSB_3_container > ul > li")

test = contents[0]
print(test.select_one('p.result_details').text, test.select_one('i.pin_generalDT').text)
print(test['data-name'],'/' ,test['data-lat'], '/' ,test['data-long'])



for c in contents:
    result = (c['data-name'], c.select_one('p.result_details').text, c.select_one('i')['class'][0], c['data-lat'],c['data-long'])
    star_list.append(result)
# print(star_list)

print("=========================")
print(star_list[0][3])



부산광역시 금정구 중앙대로 1989 (남산동)1522-3232 리저브 매장 2번
금정남산동DT / 35.2621786292157 / 129.091325043351
35.2621786292157


In [84]:
import mysql.connector
try:
    conn1 = mysql.connector.connect(
        host='localhost', # MySQL 서버 주소
        user='root',
        password='1234'
    )
    conn2 = mysql.connector.connect(
    host='210.119.14.56', # MySQL 서버 주소
    user='root',
    password='1234'
    )
    
    if conn1.is_connected():
        print("MySQL 서버1에 성공적으로 연결되었습니다.")
    if conn2.is_connected():
        print("MySQL 서버2에 성공적으로 연결되었습니다.")
except mysql.connector.Error as err:
    print("MySQL 연결 실패: ")
finally :
    if all([conn1.is_connected(), conn2.is_connected()]):
        print("연결중...")
# 커서 생성
cursor1 = conn1.cursor()
cursor2 = conn2.cursor()

MySQL 서버1에 성공적으로 연결되었습니다.
MySQL 서버2에 성공적으로 연결되었습니다.
연결중...


In [85]:
que="CREATE DATABASE IF NOT EXISTS kgy7820"
cursor1.execute(que)
cursor2.execute(que)
conn1.close()
conn2.close()
print("테이블이 생성되었습니다.")

테이블이 생성되었습니다.


In [97]:
try:
    conn1 = mysql.connector.connect(
        host='localhost', # MySQL 서버 주소
        user='root',
        password='1234',
        database="kgy7820" #사용할 데이터 베이스
    )
    conn2 = mysql.connector.connect(
    host='210.119.14.56', # MySQL 서버 주소
    user='root',
    password='1234',
    database="kgy7820" # 사용할 데이터 베이스
    )
    cursor1 = conn1.cursor()
    cursor2 = conn2.cursor()
    
    if conn1.is_connected():
        print("MySQL 서버1에 성공적으로 연결되었습니다.")
    if conn2.is_connected():
        print("MySQL 서버2에 성공적으로 연결되었습니다.")
except mysql.connector.Error as err:
    print("MySQL 연결 실패: ")
finally :
    if all([conn1.is_connected(), conn2.is_connected()]):
        print("연결중...")

MySQL 서버1에 성공적으로 연결되었습니다.
MySQL 서버2에 성공적으로 연결되었습니다.
연결중...


In [98]:
query = "CREATE TABLE IF NOT EXISTS kgy(id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(100), detail VARCHAR(100), type VARCHAR(100), lat DECIMAL(17, 14), lng  DECIMAL(17, 14))"
cursor1.execute(query)
cursor2.execute(query)
print("테이블 생성완료")

테이블 생성완료


In [99]:
insert_sql = "INSERT INTO kgy (name, detail, type ,lat, lng) VALUES (%s, %s, %s, %s, %s)"

for row in star_list:
    try:
        cursor1.execute(insert_sql, (row[0], row[1], row[2], row[3], row[4]))
        cursor2.execute(insert_sql, (row[0], row[1], row[2], row[3], row[4]))
    except Exception as e:
        print("삽입 중 에러 발생:", e)
        continue

# 커밋
conn1.commit()
conn2.commit()
print("모든 매장 데이터 삽입 완료")

모든 매장 데이터 삽입 완료
